In [1]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv\data\SI_DB_Import.xlsx"
df = pd.read_excel(file, "ENTITY")[3:].fillna('False')

In [2]:
failed_sites = []
for i, row in df.iterrows():
    temp_item = None
    temp_site = None
    if row['*SiteID'] != 'False':
        try:
            temp_site = Site.objects.get(identifier=row['*SiteID'])
        except ObjectDoesNotExist:
            failed_sites.append(row['*SiteID'])
    if temp_site:
        if row['*Entity ID'] != 'False':
            temp_item, _ = ArchEnt.objects.get_or_create(identifier=row['*Entity ID'])
            temp_item.site_id = temp_site
            
            if row['Entity Name'] != 'False':
                temp_item.name = row['Entity Name']
            
            if row['Entity type'] != 'False':
                 if row['*SiteID'] != 'False':
                    enttype = ";".join(
                        [
                            row['Entity type'],
#                             row['funerary site Type'],
#                             row['Transport Type'],
#                             row['Extraction Area Type'],
#                             row['Industrial Type'],
#                             row['ritual site'],
                            row['Deposit Type'],
                            row['Single Find Type'],

                        ]
                    )
                    items = [x for x in enttype.split(';') if x != 'False']
                    if items:
                        try:
                            item = SkosConcept.objects.get(pref_label=items[-1])
                        except ObjectDoesNotExist:
                            item = None
                        if item:
                            temp_item.ent_type = item
                            temp_item.save()
            
            if row['Topography'] != 'False':
                x = None
                for x in row['Topography'].split(','):
                    x = x.strip()
                    try:
                        y = SkosConcept.objects.get(pref_label=x)
                    except ObjectDoesNotExist:
                        y = None
                    if y:
                        temp_item.topography.add(y)
            
            if row['Burial Type'] != 'False':
                item = None
                x = row['Burial Type'].strip()
                try:
                    item = SkosConcept.objects.get(pref_label=x)
                except ObjectDoesNotExist:
                    item = None
                if item:
                    temp_item.burial_type = item
                    temp_item.save()
            
#             if row['Burial Construction'] != 'False':
#                 x = None
#                 for x in row['Burial Construction'].split(','):
#                     x = x.strip()
#                     try:
#                         y = SkosConcept.objects.get(pref_label=x)
#                     except ObjectDoesNotExist:
#                         y = None
#                     if y:
#                         temp_item.burial_construction.add(y)
            
            
            if row['Settlement Enclosure Type'] != 'False':
                x = None
                for x in row['Settlement Enclosure Type'].split(','):
                    x = x.strip()
                    try:
                        y = SkosConcept.objects.get(pref_label=x)
                    except ObjectDoesNotExist:
                        y = None
                    if y:
                        temp_item.settlement_fortification.add(y)
            
#             if row['settlement  occupation'] != 'False':
#                 item = None
#                 x = row['settlement  occupation'].strip()
#                 try:
#                     item = SkosConcept.objects.get(pref_label=x)
#                 except ObjectDoesNotExist:
#                     item = None
#                 if item:
#                     temp_item.settlement_occupation = item
#                     temp_item.save()
            
            if row['Entity Type Certainty'] != 'False':
                temp_item.type_certainty = row['Entity Type Certainty']

            if row['Dating certainty'] != 'False':
                temp_item.dating_certainty = row['Dating certainty']
            
            if row['Location certainty'] != 'False':
                temp_item.location_certainty = row['Location certainty']

            if row['Comment'] != 'False':
                temp_item.comment = row['Comment']
            
            temp_item.save()
    

In [ ]:
# ArchEnt.objects.all().delete()